In [ ]:
import pandas as pd
from sklearn.utils import shuffle
import random as rand
import numpy as np

In [ ]:
random = 24

In [ ]:
data = pd.read_csv('games.csv')
data = data[["Metacritic", "Indie", "Presence", "Platform", 
             "Memory", "RatingsBreakdown", "ReleaseDate", "Soundtrack", 
             "OriginalCost", "DiscountedCost", "Controller",
             "Achievements", "Languages", "Storage", "Tags"]]

In [ ]:
data.head()

In [ ]:
rand.seed(random)
x = [y for y in range(len(data))]
x = rand.sample(x, 1000)
data = data.iloc[x]

In [ ]:
data

In [ ]:
data.loc[:,'Storage'] = data[data['Storage'].notna()]['Storage'].apply(lambda x: x.replace('MB', ''))
data.loc[:,'Indie'] = data[data['Indie'].notna()]['Indie'].apply(lambda x: x == 1.0)
data.head()

In [ ]:
data['Storage'][0].replace('GB', '')